Folosim un alt llm pentru a evalua cererile generate anterior.


In [ ]:
!pip install -U "jax[tpu]" jaxlib flax transformers sentencepiece



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.9 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.7.2
    Uninstalling jaxlib-0.7.2:
      Successfully uninstalled jaxlib-0.7.2
  Attempting uninstall: jax
    Found existing installation: jax 0.7.2
    Uninstalling jax-0.7.2:
      Successfully uninstalled jax-0.7.2
  Attempting uninstall: flax
    Found existing installation: flax 0.10.7
    Uninstalling flax-0.10.7:
      Successfully uninstalled flax-0.10.7


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "OpenLLM-Ro/RoLlama3-8b-Instruct-DPO"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

Crearea unui promt care sa indice clar modul in care vor fi evaluate cererile generate pana acum.

In [ ]:
def evalueaza_raspuns_juridic(response_text):
    prompt_eval = f"""
Evaluează următorul text generat de model:

Text generat: \"\"\"{response_text}\"\"\"

1. Respectarea formatului: Textul respectă structura unei cereri de chemare în judecată? (da/nu + comentariu scurt)
2. Coerența: Textul este logic, coerent și respectă cerințele menționate în prompt? (da/nu + comentariu scurt)

Răspunde textual, nu în JSON.
"""
    inputs = tokenizer(prompt_eval, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=300)
    evaluare_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(evaluare_text)
    return evaluare_text


Generam cate o evaluare pentru fiecare inregistrare din cereri_transformate.json

Salvam raspunsurile intr-un alt fisier pentru a avea acces la ele mai usor.

In [ ]:
import json

with open("/content/cereri_transformate.json", "r", encoding="utf-8") as f:
    date = json.load(f)

evaluari = []

for entry in date:
    response_text = entry["raspuns_generat"]
    evaluare = evalueaza_raspuns_juridic(response_text)

    evaluari.append({
        "prompt": entry["prompt"],
        "response": response_text,
        "evaluare": evaluare
    })

with open("evaluari_responses_textuale.json", "w", encoding="utf-8") as f:
    json.dump(evaluari, f, ensure_ascii=False, indent=2)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Evaluează următorul text generat de model:

Text generat: """# CHEMARĂ ÎN JUDECĂTĂ

## Preambul

În urma evenimentelor care au avut loc, și având în vedere că există o necesitate de a se soluționa un conflict juridic, am dorit să depunem această cerere de chemare în judecată.

## Corpul documentului

1. **Parti implicate**
   - **Cel care depune cererea** (persoană fizică): Nume, prenume, adresă, C.N.I.
   - **Cel pe care se depune cererea** (persoană fizică): Nume, prenume, adresă

2. **Context/Scop**
   - **Soluționare conflict**: Se solicită judecătorului să se pronunțe asupra conflictului juridic existent între cele două persoane fizice.

3. **Conflict juridic**
   - Nu este cazul unui conflict juridic.

4. **Argumente și probe**
   - Se anexează înscrisurile justificative care susțin cererea depusă.

## Semnături

______________________________
(Nume, prenume)
(C.N.I.)

## Dată

______________________________
(Data)

## Ștampile (dacă este cazul)

______________________________
(

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Evaluează următorul text generat de model:

Text generat: """**Titlu:** Recurs în instanța judecătorească de apel

**Preambul:**

În urma examinării cererii de chemare în judecată pe care am depus-o în instanța judecătorească de primă instanță, am constatat că decizia judecătorului nu este în conformitate cu legislația aplicabilă.

**Corpus documentului:**

1. **Introducere**

În acest recurs, [Nume persoană juridică], persoană juridică, se adresează instanței judecătorești de apel, solicitând revizuirea deciziei judecătorului din instanța de primă instanță în cauza cu numărul [număr cauză].

2. **Fapte**

[Nume persoană juridică] a depus cererea de chemare în judecată în instanța de primă instanță, cu scopul de a obține o reglementare a conflictului juridic care există între noi și [Nume persoană fizică sau juridică].

3. **Argumente**

În acest context, am invocat prevederile art. 12, alin. (1) și (2) din Legea..., care stipulează că..., și art. 15, alin. (3) din Legea..., care prev

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...


Evaluează următorul text generat de model:

Text generat: """# CHEMARĂ IN JUDECĂTĂ

## Preambul

Citind cu atenție legea în vigoare, precum și regulamentele juridice aplicate, și având în vedere că există un conflict juridic în urma faptelor următoare, cerem să fim auzite în acest judecătorat, pentru a fi judecată în acest proces civil.

## Corpul documentului

### Articolul I

În urma faptelor următoare, persoana juridică [Nume persoanei juridice] (acumulativ) este chemată în judecată de persoana juridică [Nume persoanei juridice] (acumulativ), pentru a fi judecată în urma conflictului juridic existent.

### Articolul II

Conflictul juridic se referă la [Descrierea conflictului juridic].

### Articolul III

În acest caz, persoana juridică [Nume persoanei juridice] (acumulativ) este de acord să prezinte următoarele probe anexate:

1. [Descrierea probei anexate 1]
2. [Descrierea probei anexate 2]
3. [Descrierea probei anexate 3]

### Articolul IV

Persoana juridică [Nume persoanei juri

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Evaluează următorul text generat de model:

Text generat: """Chemare în judecată

Preambul

Aceasta este cererea juridică a lui [Numele persoanei fizice], cu domiciliul stabilit la [adresa domiciliului], care este implicată într-un conflict juridic cu [Numele persoanei fizice sau juridice], cu domiciliul stabilit la [adresa domiciliului].

Corpul documentului

1. În urma conflictului juridic care a apărut între noi, [Numele persoanei fizice] ceră să fie judecată în instanță.
2. [Numele persoanei fizice] dovedește că există o reglementare în forță care îi revine dreptul asupra obiectului în litigiu.
3. [Numele persoanei fizice] ceră ca [Numele persoanei fizice sau juridice] să fie obligat să restituie obiectul în litigiu sau să plătească o indemnizație echivalentă.
4. [Numele persoanei fizice] a anexat înscrisurile justificative necesare pentru dovadă a drepturilor sale asupra obiectului în litigiu.
5. [Numele persoanei fizice] ceră ca instanța să se pronunțe în favoarea sa până la pri

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Evaluează următorul text generat de model:

Text generat: """**CHEMARE ÎN JUDECĂTĂ**

**Titlu:** Chemare în judecată

**Preambul:**

În urma conflictului juridic care a apărut între [Numele persoanei juridice/ numele persoanei fizice] („Partida 1”) și [Numele persoanei juridice/ numele persoanei fizice] („Partida 2”) în legătură cu [contextul sau scopul cererii], Partida 1 este nevoită să depună acest act de chemare în judecată pentru a obține o dovadă a drepturilor sale.

**Corpul documentului:**

1. Partida 1 este o persoană juridică cu sediul în [adresa sediului], înregistrată în Registrul de Comerț cu numărul [numărul înregistrării].
2. Partida 2 este o persoană juridică cu sediul în [adresa sediului], înregistrată în Registrul de Comerț cu numărul [numărul înregistrării].
3. În urma unei transacții contractuale încheiate între Partida 1 și Partida 2, există un conflict juridic în legătură cu [detalii despre conflictul juridic].
4. În acest sens, Partida 1 invocă prevederile artic

In [ ]:
system_prompt = """
Tu ești un generator de documente juridice în limba română.

Creezi modele generale de cereri judiciare, fără consultanță juridică individualizată.
Redactezi exclusiv texte scurte, de maximum 200 de cuvinte.

Reguli stricte:
– Creezi doar șabloane generale, nu soluții sau strategii juridice.
– Folosești formulări neutre și exemple generice (fără date reale).
– Structura documentului trebuie să fie EXACT următoarea:

1. TITLU
2. PĂRȚILE IMPLICATE
3. CAUZA
4. CONFLICTUL
5. DATA
6. SEMNĂTURA
7. ȘTAMPILA (doar dacă este cazul; altfel menționează „Nu se aplică”)

În fiecare secțiune folosește un paragraf scurt și clar.
Nu depăși 200 de cuvinte în total.

Așteaptă tipul cererii și generează un model general.

"""


In [ ]:
def generate_document(request_type: str, max_tokens=600):
    """
    request_type = tipul cererii (ex.: 'cerere de chemare în judecată')
    """
    prompt = system_prompt + f"\n\nUtilizator: Generează un model general de {request_type}.\nAsistent:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.3,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.1
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
print(generate_document("Cerere de chemare în judecată"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Tu ești un generator de documente juridice în limba română.

Creezi modele generale de cereri judiciare, fără consultanță juridică individualizată. 
Redactezi exclusiv texte scurte, de maximum 200 de cuvinte.

Reguli stricte:
– Creezi doar șabloane generale, nu soluții sau strategii juridice.
– Folosești formulări neutre și exemple generice (fără date reale).
– Structura documentului trebuie să fie EXACT următoarea:

1. TITLU  
2. PĂRȚILE IMPLICATE  
3. CAUZA  
4. CONFLICTUL  
5. DATA  
6. SEMNĂTURA  
7. ȘTAMPILA (doar dacă este cazul; altfel menționează „Nu se aplică”)

În fiecare secțiune folosește un paragraf scurt și clar. 
Nu depăși 200 de cuvinte în total.

Așteaptă tipul cererii și generează un model general.



Utilizator: Generează un model general de Cerere de chemare în judecată.
Asistent: Titlu: Cerere de chemare în judecată\n\nPărțile implicate: Plângerea [Numele Pârâtului]\n\nCauza: Contractul din data de [Data] între Plângerea și Pârâtul pentru furnizarea de bunuri/serv